### How to query the SDMX global registry for DSDs and codelists:
**ValidatFOSS2** project\
Olav ten Bosch, Mark van der Loo, Statistics Netherlands\
20210513

In [11]:
import requests                  # for issueing HTTP requests
import json                      # for processing json data

- Viewer: https://registry.sdmx.org/overview.html
- Web service playground: https://registry.sdmx.org/webservice/structure.html

In [12]:
# Settings for the SDMX global registry API:
endpoint = 'https://registry.sdmx.org/ws/public/sdmxapi/rest/'

# Note: the 'format' querystring parameter that we use below is not standard SDMX 2.1 but a convenience from registry.sdmx.org

In [13]:
# Some utility functions to select the elements useful for validation:
def urn_short(urn):
    return urn.partition("=")[2]
    
def DSD_el(el):
    s = urn_short(el['urn'])
    sep = ' '
    
    representation = el['representation']
    if "textFormat" in representation:
        textformat = representation['textFormat']
        s += sep+textformat['textType']
        if "maxLength" in textformat:
            s += sep+str(textformat['maxLength'])
    elif "representation" in el:
        s += sep+urn_short(representation['representation'])
    else:
        s += sep+el['id']
        
    if "mandatory" in el and el["mandatory"] == True: 
        s += sep+'mandatory'
    return s

In [14]:
# Retrieve a specific codelist in json:
resource = 'codelist'
agencyID = 'ESTAT'
resourceID = 'CL_ACTIVITY'
version = 'latest'  # 20210508: results in'version '1.7'

r1 = requests.get(f"{endpoint}{resource}/{agencyID}/{resourceID}/{version}", params = {'format': 'sdmx-json'})
print(r1.url, r1.status_code)
#print(r1.text)

# Print codelist name, version and all codes and names:
codelist = json.loads(r1.text)["Codelist"][0]
print(urn_short(codelist['urn']))
for item in codelist['items']:
    print(item['id'], item['names'][0]['value'])


https://registry.sdmx.org/ws/public/sdmxapi/rest/codelist/ESTAT/CL_ACTIVITY/latest?format=sdmx-json 200
ESTAT:CL_ACTIVITY(1.7)
_T Total - All activities
_X Unspecified
_Z Not applicable
A Agriculture, forestry and fishing
A_B Total Primary sector
A01 Crop and animal production, hunting and related service activities
A02 Forestry and logging
A03 Fishing and aquaculture
ATS All activities except activities of households as employers; undifferentiated goods- and services-producing activities of households for own use and activities of extraterritorial organisations and bodies
ATUXFPRV Activities other than financial and insurance
B Mining and quarrying
B05 Mining of coal and lignite
B06 Extraction of crude petroleum and natural gas
B06_09 Extraction of crude petroleum and natural gas; mining support service activities
B07 Mining of metal ores
B08 Other mining and quarrying
B09 Mining support service activities
BTE Industry (except construction)
BTF Industry and construction
BTN Business e

In [15]:
# Retrieve same codelist in xml (sdmx-ml):
resource = 'codelist'
agencyID = 'ESTAT'
resourceID = 'CL_ACTIVITY'
version = 'latest'  # 20210508: results in'version '1.7'

r2 = requests.get(f"{endpoint}{resource}/{agencyID}/{resourceID}/{version}")
print(r2.url, r2.status_code)
print(r2.text)

https://registry.sdmx.org/ws/public/sdmxapi/rest/codelist/ESTAT/CL_ACTIVITY/latest 200
<?xml version='1.0' encoding='UTF-8'?><mes:Structure xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:mes="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:str="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure" xmlns:com="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common" xsi:schemaLocation="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message https://registry.sdmx.org/schemas/v2_1/SDMXMessage.xsd"><mes:Header><mes:ID>IREF620809</mes:ID><mes:Test>false</mes:Test><mes:Prepared>2021-05-12T23:28:01Z</mes:Prepared><mes:Sender id="SDMX" /><mes:Receiver id="not_supplied" /></mes:Header><mes:Structures><str:Codelists><str:Codelist urn="urn:sdmx:org.sdmx.infomodel.codelist.Codelist=ESTAT:CL_ACTIVITY(1.7)" isExternalReference="false" agencyID="ESTAT" id="CL_ACTIVITY" isFinal="true" version="1.7"><com:Name xml:la

In [16]:
# Retrieve a specific DSD (specific version or the latest) in SDMX-JSON:
resource = 'datastructure'
agencyID = 'ESTAT'
resourceID = 'CPI'
version = 'latest' # 20210508: results in version '1.2'

r3 = requests.get(f"{endpoint}{resource}/{agencyID}/{resourceID}/{version}", params = {'format': 'sdmx-json'})
print(r3.url, r3.status_code)
#print(r3.text)

# Print some elements of DSD important for validation:
structure = json.loads(r3.text)["DataStructure"][0]
print(urn_short(structure['urn']))
print('primaryMeasure:')
print(DSD_el(structure['primaryMeasure']))
print('Dimensions and codelists:')
for dimension in structure['dimensionList']['dimensions']:
    print(DSD_el(dimension))
print('Attributes:')
for attribute in structure['attributeList']['attributes']:
    print(DSD_el(attribute))

https://registry.sdmx.org/ws/public/sdmxapi/rest/datastructure/ESTAT/CPI/latest?format=sdmx-json 200
ESTAT:CPI(1.2)
primaryMeasure:
ESTAT:CPI(1.2).OBS_VALUE Double
Dimensions and codelists:
ESTAT:CPI(1.2).FREQ SDMX:CL_FREQ(2.0)
ESTAT:CPI(1.2).SEASONAL_ADJUST ESTAT:CL_SEASON_ADJUST(1.0)
ESTAT:CPI(1.2).REF_AREA IMF:CL_AREA(1.13)
ESTAT:CPI(1.2).IND_TYPE ESTAT:CL_IND_TYPE(1.1)
ESTAT:CPI(1.2).IDX_TYPE ESTAT:CL_IDX_TYPE(1.0)
ESTAT:CPI(1.2).BASE_PER ObservationalTimePeriod
ESTAT:CPI(1.2).ITEM ESTAT:CL_ITEM_PRICE(1.1)
ESTAT:CPI(1.2).COVERAGE_GEO ESTAT:CL_COVERAGE_GEO(1.0)
ESTAT:CPI(1.2).COVERAGE_POP ESTAT:CL_COVERAGE_POP(1.0)
ESTAT:CPI(1.2).UNIT_MEASURE IMF:CL_UNIT(1.14)
ESTAT:CPI(1.2).TRANSFORMATION ESTAT:CL_TRANSFORMATION(1.4)
ESTAT:CPI(1.2).CUST_BREAKDOWN ESTAT:CL_CUST_BREAKDOWN(1.0)
ESTAT:CPI(1.2).TIME_PERIOD ObservationalTimePeriod
Attributes:
ESTAT:CPI(1.2).DISS_ORG IMF:CL_ORGANISATION(1.10)
ESTAT:CPI(1.2).COMPILING_ORG IMF:CL_ORGANISATION(1.10)
ESTAT:CPI(1.2).DATA_COMP String 4000
ESTAT

In [17]:
# Retrieve a specific DSD (specific version or the latest) in XML (SDMX-ML):
resource = 'datastructure'
agencyID = 'ESTAT'
resourceID = 'CPI'
version = 'latest' # 20210508: results in version '1.2'

r4 = requests.get(f"{endpoint}{resource}/{agencyID}/{resourceID}/{version}")
print(r4.url, r4.status_code)
print(r4.text)


https://registry.sdmx.org/ws/public/sdmxapi/rest/datastructure/ESTAT/CPI/latest 200
<?xml version='1.0' encoding='UTF-8'?><mes:Structure xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:mes="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:str="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure" xmlns:com="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common" xsi:schemaLocation="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message https://registry.sdmx.org/schemas/v2_1/SDMXMessage.xsd"><mes:Header><mes:ID>IREF620811</mes:ID><mes:Test>false</mes:Test><mes:Prepared>2021-05-12T23:28:08Z</mes:Prepared><mes:Sender id="SDMX" /><mes:Receiver id="not_supplied" /></mes:Header><mes:Structures><str:DataStructures><str:DataStructure urn="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=ESTAT:CPI(1.2)" isExternalReference="false" agencyID="ESTAT" id="CPI" isFinal="true" version="1.2"><com:Name xml:l